In [154]:
import pandas as pd
import numpy as np
import sklearn.metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn import svm
import random
import pickle
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset, Dataset
import torch.utils.data as data_utils
import torch.optim as optim
from torch.autograd import Variable
import sklearn.model_selection as ms

In [54]:
pc_train = pd.read_csv('pc_train.csv')
pc_test = pd.read_csv('pc_test.csv')

# I'm going to break off a validation dataset -- 1/5 of test
# and 2/15 of train, adding up to 15% of total rows.

# NOTE: (don't have time to change) I don't really need a validation
# and test set using cross-validation.

pc_train, valid1 = ms.train_test_split(pc_train, test_size = 2/15)
pc_test, valid2 = ms.train_test_split(pc_test, test_size = 1/5)
pc_valid = valid1.append(valid2)
x_train = pc_train[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'winter', 'spring', 'summer', 'fall', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec', 'Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]
y_train = pc_train['target']
x_test = pc_test[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'winter', 'spring', 'summer', 'fall', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec', 'Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]
y_test = pc_test['target']
x_valid = pc_valid[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'winter', 'spring', 'summer', 'fall', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec', 'Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]
y_valid = pc_valid['target']

# I'm going to break off a validation dataset -- 1/5 of test
# and 2/15 of train, adding up to 15% of total rows.

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_valid.shape)
print(y_valid.shape)

(22068, 30)
(22068,)
(6790, 30)
(6790,)
(5094, 30)
(5094,)


In [3]:
# Let's use mean absolute error and mean squared error as the 
# performance metrics. I will choose 5 folds to save time, 
# would normally choose 10.

reg = LinearRegression()
scores = cross_val_score(reg, x_train, y_train, cv = 5, scoring = 'neg_mean_absolute_error')
reg_score = sum(scores) / len(scores)
print(reg_score)

scores = cross_val_score(reg, x_train, y_train, cv = 5, scoring = 'neg_mean_squared_error')
reg_score = sum(scores) / len(scores)
print(reg_score)

reg_model = reg.fit(x_train, y_train)
pickle.dump(reg_model, open('models/reg_model', 'wb'))

# Well, that was the most basic score. Our future models
# beat this value. Since this is the negative mean absolute
# error, we are hunting for values closer to 0 than -0.585.

gbr = GradientBoostingRegressor(loss = 'ls', n_estimators = 300)
scores = cross_val_score(gbr, x_train, y_train, cv = 5, scoring = 'neg_mean_absolute_error')
gbr_score = sum(scores) / len(scores)
print(gbr_score)

scores = cross_val_score(gbr, x_train, y_train, cv = 5, scoring = 'neg_mean_squared_error')
gbr_score = sum(scores) / len(scores)
print(gbr_score)

gbr_model = gbr.fit(x_train, y_train)
pickle.dump(gbr_model, open('models/gbr_model', 'wb'))

# This is an improvement. The mean squared error decreased by 0.02.



-0.5858349483211118
-0.5950358465524243
-0.5654723540527746
-0.5706264836450041


In [4]:
# This section can take quite a while to run.

eln = ElasticNet(alpha = 0.2, max_iter = 10)
scores = cross_val_score(eln, x_train, y_train, cv = 5, scoring = 'neg_mean_absolute_error')
eln_score = sum(scores) / len(scores)
print(eln_score)

eln = ElasticNet(alpha = 0.2, max_iter = 10)
scores = cross_val_score(eln, x_train, y_train, cv = 5, scoring = 'neg_mean_squared_error')
eln_score = sum(scores) / len(scores)
print(eln_score)

eln_model = eln.fit(x_train, y_train)
pickle.dump(eln_model, open('models/eln_model', 'wb'))

# Well, this is no good. The mean absolute error decreases
# as alpha approaches 0, but at 0, the elastic net becomes
# a simple linear regression.

rfr = RandomForestRegressor(n_estimators = 200)
scores = cross_val_score(rfr, x_train, y_train, cv = 5, scoring = 'neg_mean_absolute_error')
rfr_score = sum(scores) / len(scores)
print(rfr_score)

rfr = RandomForestRegressor(n_estimators = 200)
scores = cross_val_score(rfr, x_train, y_train, cv = 5, scoring = 'neg_mean_squared_error')
rfr_score = sum(scores) / len(scores)
print(rfr_score)

# The random forest model performs well but takes
# a while to build the models. I think adding more
# predictors would probably improve the results. I will
# do that when I evaluate the models against the validation
# results.

rfr = RandomForestRegressor(n_estimators = 400)
rfr_model = rfr.fit(x_train, y_train)
pickle.dump(rfr_model, open('models/rfr_model', 'wb'))


-0.6145246590167524
-0.6405849252292314
-0.568586235456171
-0.5735189409605741


In [158]:
def outputSize(in_size, kernel_size, stride, padding):
    output = int((in_size - kernel_size + 2*(padding)) / stride) + 1
    return(output)

class MyMLP(nn.Module):
    def __init__(self):
        super(MyMLP, self).__init__()
        self.hidden1 = nn.Linear(30, 22)
        self.hidden2 = nn.Linear(22, 3)
        #self.hidden3 = nn.Linear(20, 16)
        #self.hidden4 = nn.Linear(16, 12)
        #self.hidden5 = nn.Linear(12, 8)
        self.out = nn.Linear(3, 1)
        #self.hidden = nn.Linear(178, 16)
        #self.out = nn.Linear(16, 5)

    def forward(self, x):
        sigmoid = torch.nn.Sigmoid()
        x = nn.functional.relu(self.hidden1(x))
        x = nn.functional.relu(self.hidden2(x))
        #x = nn.functional.relu(self.hidden3(x))
        #x = nn.functional.relu(self.hidden4(x))
        #x = nn.functional.relu(self.hidden5(x))
        x = sigmoid(self.out(x))

        return x

# I didn't have enough time to configure and implement
# the CNN and RNN models

class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv1d(6, 16, 5)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(in_features=16 * 41, out_features=256)
        #self.fc2 = nn.Linear(128, 5)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, 16)
        self.fc5 = nn.Linear(16, 5)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 41)
        x = nn.functional.relu(self.fc1(x))
        #x = self.fc2(x)
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.relu(self.fc3(x))
        x = nn.functional.relu(self.fc4(x))
        x = self.fc5(x)
        return x
    
class MyRNN(nn.Module):
    def __init__(self):
        super(MyRNN, self).__init__()
        #self.rnn = nn.GRU(input_size=1, hidden_size=16, num_layers=2, batch_first=True, dropout=0.5)
        #self.fc = nn.Linear(in_features=16, out_features=5)
        
        self.rnn1 = nn.GRU(input_size=1, hidden_size=32, num_layers=2, batch_first=True, dropout=0.5)
        self.rnn2 = nn.GRU(input_size=32, hidden_size=128, num_layers=2, batch_first=True, dropout=0.5)
        self.fc1 = nn.Linear(in_features=128, out_features=32)
        self.fc2 = nn.Linear(in_features=32, out_features=16)
        self.fc3 = nn.Linear(in_features=16, out_features=5)

    def forward(self, x):
        #x, _ = self.rnn(x)
        #x = torch.tanh(x[:, -1, :])
        #x = self.fc(x)
        
        x, _ = self.rnn1(x)
        x = torch.tanh(x)
        x, _ = self.rnn2(x)
        x = torch.tanh(x[:, -1, :])
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


SyntaxError: invalid syntax (<ipython-input-158-5a5af783d79b>, line 35)

In [165]:
MODEL_TYPE = 'MLP'  # Change this to 'MLP' or 'CNN'
NUM_EPOCHS = 25
BATCH_SIZE = 32
USE_CUDA = False  
NUM_WORKERS = 2  # Number of threads used by DataLoader

target = Variable(torch.tensor(np.array(y_train)).float())
MLP_data = Variable(torch.tensor(np.array(x_train)).float())
MLP_data_test = Variable(torch.tensor(np.array(x_test)).float())

model = MyMLP()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion.to(device)

for epoch in range(NUM_EPOCHS):
    y_pred = model(MLP_data)
    loss = criterion(y_pred, target)
    if epoch % 5 == 0:
        print('epoch: ', epoch,' loss: ', loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print('epoch: ', epoch,' loss: ', loss.item())
#pickle.dump(model, open('models/mlp.pth', 'wb'))

# The MLP neural network doesn't seem to work well
# in this case. I'd like to try others...


epoch:  0  loss:  1.3810876607894897
epoch:  5  loss:  0.8252854347229004
epoch:  10  loss:  0.8142426609992981
epoch:  15  loss:  0.8122402429580688
epoch:  20  loss:  0.8117661476135254
epoch:  24  loss:  0.8116387724876404


In [187]:
# Normally, if I had more time, I would have implemented
# the CNN, RNN, and possibly other models. I would also
# use cross-validation for the neural networks. There 
# is not a need for a test and validation dataset but
# only a test set in cross-validation. I'm not going to
# fix that here since there is no time. My last step is
# to compare the results of the models using the test set.
# There's a bunch more that I want to do but don't have time.

reg_model
gbr_model
eln_model
rfr_model
model

print(sklearn.metrics.mean_absolute_error(y_test, reg_model.predict(x_test)))
print(sklearn.metrics.mean_absolute_error(y_test, gbr_model.predict(x_test)))
print(sklearn.metrics.mean_absolute_error(y_test, eln_model.predict(x_test)))
print(sklearn.metrics.mean_absolute_error(y_test, rfr_model.predict(x_test)))
print(sklearn.metrics.mean_absolute_error(y_test, model(Variable(torch.tensor(np.array(x_test)).float())).detach()))

mean_target = sum(y_test) / len(y_test)

print(sum([abs(mean_target - a) for a in y_test]) / len(y_test))

# It looks like on new data, it was a tie between the gbr
# and rfr models. It looks like the ensemble methods win.

# As I've said before, I ran out of time and was unable to
# implement other models, like the RNN, which I believe
# would probably improve the mean absolute error.

# On any particular day, we expect the power company would be able to
# predict the amount of active power used by this household
# to within 0.565. That is significantly better than a pure guess,
# which we would expect to be within about 0.72 of actual active power.


0.5842042271698018
0.5658239520507302
0.6130986277282414
0.5684935151327679
0.709200482834881
0.7217957967215702
